<a href="https://colab.research.google.com/github/siddharthchd/introml/blob/master/unit7/demo_computing_gradients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Computing Gradients

In [4]:
import numpy as np

#### Example 1 : A simple vector-input function

Suppose f(w) = w_0^2 + 2w_0w_1^3. Then the function below computes f(w) and its gradients

In [5]:
def feval(w):

    # function
    f = w[0] ** 2 + 2 * w[0] * (w[1] ** 3)

    # gradient
    df0 = 2 * w[0] + 2 * (w[1] ** 3)
    df1 = 6 * w[0] * (w[1] ** 2)
    fgrad = np.array([df0, df1])

    return f, fgrad

# point to evaluate
w = np.array([2, 4])
f, fgrad = feval(w)

print('f        = %f' % f)
print('fgrad    = {}'.format(fgrad))

f        = 260.000000
fgrad    = [132 192]


#### Example 2 : Non-Linear least squares for an exponential model

Consider an exponential model \\

\begin{align*}
    yhat = a * exp(-b * x) \\
\end{align*}

for parameters $w = [a, b]$. Given training data $(x[i], y[i])$ a natural loss function is given by : \\

\begin{align*}
    J(w) := \sum_i (y[i] - yhat[i])^2, \\ yhat[i] = a * exp(-b * x[i]) \\
\end{align*}


The following code computes the loss function $J(w)$ and its gradient $\frac{dJ}{dw}$


In [6]:
def Jeval(w):

    # unpack vector
    a = w[0]
    b = w[1]

    # compute the loss function
    yerr = y - a * np.exp(-b * x)
    J = 0.5 * np.sum(yerr ** 2)

    # compute the gradient
    dJ_da = -np.sum(yerr * np.exp(-b * x))
    dJ_db = np.sum(yerr * a * x * np.exp(-b * x))
    Jgrad = np.array([dJ_da, dJ_db])

    return J, Jgrad

# generate some random data
ny = 100
y = np.random.randn(ny)
x = np.random.rand(ny)

# some arbitrary parameters to compute the gradient at
a = 1
b = 2
w = np.array([a, b])

J, Jgrad = Jeval(w)
print('Jgrad = {}'.format(Jgrad))

Jgrad = [21.85747909 -4.74780678]


#### Example 3 : Loss function for a Log-Linear Model

Now suppose we have a lograithmic linear model : \\
\begin{align*}
\hat{y} & = log(z_i) \\
z & = w_0 + \sum_j w_j x_j \\
\end{align*}


If we have data X, y, the prediction on sample i is : \\

\begin{align*}
\hat{y_i} & = log(z_i) \\
z_i & = w_0 + \sum_j w_j * x_{ij} \\
\end{align*}

Suppose we use MSE loss:

\begin{align*}
J(w) = \sum_i (y_i - yhat_i )^2 \\
\end{align*}

To compute the components $\frac{dJ}{dw_j}$, first write $z = Aw$ where $A=[1 X]$, the matrix with ones on the first column. Then, $\frac{dz_i}{dw_j} = A_{ij}$. Also, $\frac{d\hat{y_i}}{dz_i} = \frac{1}{z_i}$, so with the multi-variable chain rule:

\begin{align*}
\frac{dJ}{dw_j} & = \sum_i \frac{dJ}{d \hat{y_i}} * \frac{d \hat{y_i}}{dw_j} \\
& = \sum_i (\frac{dJ}{d \hat{y_i}}) * (\frac{d \hat{y_i}}{dz_i}) * (\frac{dz_i}{dw_j})\\
& = 2 * \sum_i (\hat{y_i} - y_i) * \frac{1}{z_i} * A_{ij} \\
\end{align*}

We can implement the loss and gradient computation as follows:


In [7]:
def  Jeval(w, X, y):

    # Create matrix A = [1, X]
    n = X.shape[0]
    A = np.column_stack((np.ones(n), X))

    # Compute function
    z = A.dot(w)
    yhat = np.log(z)
    J = np.sum((y - yhat) ** 2)

    # Compute gradient
    dj_dz = 2 * (yhat - y) / z
    Jgrad = A.T.dot(dj_dz)

    return J, Jgrad

Whenever you implement a gradient you should always check the gradients. Errors in the gradient are the number 

- Take two points w0 and w1 close to one another
- Verify that J(w1) - J(w0) is close to Jgrad(w0).dot(w1 - w0)

In [12]:
# Generate random positive data
n = 100
d = 5
X = np.random.uniform(0, 1, (n, d))
w0 = np.random.uniform(0, 1, (d+1, ))
y = np.random.uniform(0, 2, (n, ))

# Compute function and gradient at point w0
J0, Jgrad0 = Jeval(w0, X, y)

# Take a small pertubation
step = 1e-4
w1 = w0 + step * np.random.normal(0, 1, (d+1,))

# Evaluate the function at perterbed point
J1, Jgrad1 = Jeval(w1, X, y)

dJ= J1 - J0
dJ_est = Jgrad0.dot(w1 - w0)
print('Actual differnece :  %12.4e' % dJ)
print('Estimated differnece :  %12.4e' % dJ_est)

Actual differnece :   -1.7047e-02
Estimated differnece :   -1.7050e-02


#### Example 3 : A function of a matrix

Suppose $f(W) = a'*W*b$. Then, $ \partial f(W) = a*b.T$

In [13]:
def feval(W, a, b):

    # Function
    f = a.dot(W.dot(b))

    # Gradient -- using python broadcasting
    fgrad = a[:, None] * b[None, :]

    return f, fgrad

# Some random data
m = 4
n = 3
W = np.random.randn(m, n)
a = np.random.randn(m)
b = np.random.randn(n)

f, fgrad = feval(W, a, b)